In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
weather_data = pd.read_csv("Resources/Weather Data/GlobalLandTemperaturesByState.csv")

weather_data.head()

In [ ]:
# Drop unnecessary columns
weather_data = weather_data.drop(weather_data["AverageTemperatureUncertainty"])

In [ ]:
# change the dt column to datetime
weather_data["dt"] = pd.to_datetime(weather_data["dt"], format="%Y-%m-%d")
weather_data

In [ ]:
#only selecting data from the United States and from 1900 and on
us_weather = weather_data.loc[weather_data["Country"] == "United States"]
us_weather = us_weather.loc[us_weather["dt"] > "1900-01-01"]


#selecting only the data from california, florida, and illinois
ca_weather = us_weather.loc[us_weather["State"] == "California"].copy()
fl_weather = us_weather.loc[us_weather["State"] == "Florida"].copy()
il_weather = us_weather.loc[us_weather["State"] == "Illinois"].copy()

In [ ]:
#concatenating the data together into one dataframe
frames = [ca_weather, fl_weather, il_weather]

state_weather = pd.concat(frames)

In [ ]:
# adding a column for the Average Temperature in Degrees Fahrenheit
state_weather["Average Temperature (F)"] = (state_weather["AverageTemperature"] * (9 / 5)) + 32

# converting float type columns to integers in order to round to the nearest whole number
state_weather['AverageTemperature'] = state_weather['AverageTemperature'].astype(np.int64)
state_weather['Average Temperature (F)'] = state_weather['Average Temperature (F)'].astype(np.int64)

In [ ]:
#Rename Column Headers
state_weather.rename(columns={"dt": "Date", "AverageTemperature":"Average Temperature (c)"})